In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys

In [2]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-09-24 10:49:25.378569


In [3]:
HTCondorRun = str(sys.argv[1])
print("HTCondorRun:",HTCondorRun)

HTCondorRun: -f


In [4]:
save_data = False
tmp_data = False
take_subset = True
subset_size = 100

In [5]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "Dstlnu_skim_BsBtag_separation_{}_evts/".format(subset_size)   
root_subdir = "axheim_DstlnuSKIM_run1/"   

root_path = nfs_path + "Dstlnu_skim/" + root_subdir

In [6]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"

In [7]:
fileY4S = uproot.open(root_path + merged + "DXtagDstl.root:variables")

In [ ]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename = root_path + merged + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/Dstlnu_skim/axheim_DstlnuSKIM_run1/merged_gammas.root:variables


In [ ]:
df_FSPs = pd.concat(dfs)

In [ ]:
df_Y4S = fileY4S.arrays(library="pd")

In [ ]:
print(df_FSPs.shape[0])


In [ ]:
# delete FSPs for which no Y4S file entry was found
#df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [ ]:
print(df_FSPs.shape[0])


## take a sample if used in notebook for faster processing

In [ ]:
all_evt_nums = df_FSPs['__event__'].unique()
all_evt_nums.shape[0]

In [ ]:
sample_evt_nums = np.random.choice(all_evt_nums, size=subset_size)
sample_evt_nums.shape[0]

In [ ]:
if take_subset:
    df_FSPssample = df_FSPs[df_FSPs['__event__'].isin(sample_evt_nums)]
    #df_Y4S=df_Y4Ssample
    df_FSPs=df_FSPssample

In [ ]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("numEvents:",df_FSPs['__event__'].unique().shape[0])

### delete particles which occur more than ones based on uniqueParticleIdentifier

In [ ]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsFSPs_uniqParID.sort_values("count"))

In [ ]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("groupsFSPs_uniqParID.shape[0]:",groupsFSPs_uniqParID.shape[0])
print("df_Y4S.shape[0]:",df_Y4S.shape[0])

In [ ]:
# delete particles which occur more than ones (keep first) and if possible keep the one with basf2_used==1
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())
df_FSPs = df_FSPs.sort_values("basf2_used",ascending=False).drop_duplicates(subset=("__event__","uniqueParticleIdentifier"), keep='first')
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())

In [ ]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])

## save sample 

In [ ]:
if take_subset:
    df_FSPs.to_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S.to_csv(root_path + "df_Y4S_sample__evts.csv")

In [ ]:
load = False
if load:
    df_FSPs = pd.read_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S = pd.read_csv(root_path + "df_Y4S_sample10evts.csv")

### print one event

In [ ]:
df_FSPs["__event__"]

In [ ]:
df_FSPs[df_FSPs["__event__"] == 6620998].sort_values('mcPDG')[['mcPDG','genMothPDG_0', 'mcMother0_uniqParID', 'genMotherID_0',
'genMothPDG_1', 'mcMother1_uniqParID', 'genMotherID_1',
'genMothPDG_2', 'mcMother2_uniqParID', 'genMotherID_2',
'genMothPDG_3', 'mcMother3_uniqParID', 'genMotherID_3',
'genMothPDG_4', 'mcMother4_uniqParID', 'genMotherID_4',
'genMothPDG_5', 'mcMother5_uniqParID', 'genMotherID_5',
'genMothPDG_6', 'mcMother6_uniqParID', 'genMotherID_6',
'genMothPDG_7', 'mcMother7_uniqParID', 'genMotherID_7',
'genMothPDG_8', 'mcMother8_uniqParID', 'genMotherID_8',
'genMothPDG_9', 'mcMother9_uniqParID', 'genMotherID_9']]

## check if category combinations make sense

In [ ]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

### add B_ID col with extra info

In [ ]:
# function to create col with the particles mother B's uniqueParticleIdentifier
def B_ID(s):
    label = 0
    for i in range(10):
        mcMotheri_uniqParID = "mcMother{}_uniqParID".format(i)
        if ((s[mcMotheri_uniqParID]) == 83886082.0):
            label = 83886082   
        elif ((s[mcMotheri_uniqParID]) == 83886081.0):
            label = 83886081   
    return label
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

In [ ]:
# function to create col with info which i-th daughter of B the particle is (0=daughter,1=granddaughter,.. -1=no B daughter)
#def B_iDaughter(s):
#    label = -1
#    for i in range(10):
#        mcMotheri_uniqParID = "mcMother{}_uniqParID".format(i)
#        if ((s[mcMotheri_uniqParID] == 83886082.0) or (s[mcMotheri_uniqParID] == 83886081.0)):
#            label = i   
#            return label
#    return label
#df_FSPs['B_iDaughter'] = df_FSPs.apply(B_iDaughter, axis=1)

In [ ]:
#df_FSPs["mcPDG_abs"] = df_FSPs["mcPDG"].abs()
#mcPDG_BID_combis = pd.DataFrame({'count' : df_FSPs.groupby( ["mcPDG_abs","B_ID","B_iDaughter"] ).size()}).reset_index()


#printDF = mcPDG_BID_combis[mcPDG_BID_combis["B_iDaughter"]==0].sort_values('B_ID',ascending=False)

#pd.set_option('display.float_format', lambda x: '%.0f' % x)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(printDF)

In [ ]:
# this shows that Hc is sometimes combined from both B's which is of course wrong
#groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","B_ID","Hc","basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsAllFSPs.sort_values("Hc"))

## data preprocessing

### save dataframes on NFS

In [ ]:
if HTCondorRun == "isHTCondorRun":
    df_FSPs.to_csv(root_path + "df_FSPs_preProcessed_{}_evts.csv".format(subset_size))
    

### load dataframes

In [ ]:
#df_FSPs = pd.read_csv(root_path + "df_FSPs_preProcessed.csv")
#event_Bs = pd.read_csv(root_path + "event_Bs.csv")

In [ ]:
df_FSPs_final = df_FSPs

# start of NN data creation

In [ ]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

In [ ]:
data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir)    
if save_data:
    data_dir.mkdir(parents=True, exist_ok=True)
print("Will save data to:", data_dir,'is', save_data ,'\n')

In [ ]:
for num_FSPs_toData in range(minFSPs, maxFSPs+1):
    df_num_subset = df_FSPs_final.copy()
    df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]
    
        
    numEvents = df_num_subset.__event__.nunique()
    print("numEvents:",numEvents)
    print("num_FSPs_toData:",num_FSPs_toData)  
    if numEvents == 0:
        print("skipped because empty \n")
        continue

    if numEvents < 10:
        print("skipped because <10 events \n")
        continue
    
    num_features = 4
    leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
    SA_target =  np.zeros((numEvents, num_FSPs_toData))
    global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)
    
    event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
    #print("len(event_list):",len(event_list))
    for i in range(numEvents):

        event_iter = event_list[i]

        global_tag_masterInfo = "evt" + str(event_iter)
        global_tag[i,-1] = global_tag_masterInfo
        #print("global_tag[i,-1]:",global_tag[i,-1])
        #print("i:",i,"event_iter:",event_iter)

        event_df = df_num_subset[df_num_subset.__event__ == event_iter]

        for j in range(num_FSPs_toData):
            #print("numParticle:",j)
            particle = event_df.iloc[j]

            #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
            leaves[i,j,0] = particle["px"]
            leaves[i,j,1] = particle["py"]
            leaves[i,j,2] = particle["pz"]
            leaves[i,j,3] = particle["E"]
            
            basf2_usage = "basf2_NONE"
            if particle["basf2_Bsig"] == 1.0:
                basf2_usage = "basf2_Bsig"
            elif particle["basf2_X"] == 1.0:
                basf2_usage = "basf2_X"
            elif particle["basf2_used"] == 0:
                basf2_usage = "basf2_bg"

            global_tag_Info = str((particle["mcPDG"])) 
            global_tag_Info += "_" + basf2_usage
            global_tag[i,j] = global_tag_Info

            label = -10 # error code if assignment fails
            B_tag_uniqID = 83886081.0
            B_sig_uniqID = 83886082.0
            if particle["B_ID"] == B_tag_uniqID:
                label = 1 # particle belongs to Btag (MC truth)
            elif particle["B_ID"] == B_sig_uniqID:
                label = 2 # particle belongs to Bsig (MC truth)
            elif particle["B_ID"] == 0:
                label = 0 # background
            
            
            SA_target[i,j] = label
            
        del event_df
        
        
    # shuffle the data    
    for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
        perms = np.random.permutation(leaves.shape[1])

        leaves[idx,:] = leaves[idx,perms]
        SA_target[idx,:] = SA_target[idx,perms]
        global_tag[idx,0:-1] = global_tag[idx,perms]
        
        
         


    #print(global_tag)
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    print("leaves.shape:",leaves.shape)
    print("SA_target.shape:",SA_target.shape)
    print("global_tag.shape:",global_tag.shape)


    print("leaves[0]:",leaves[0])
    print("SA_target[0]:",SA_target[0])
    print("global_tag[0]:",global_tag[0])

    x=leaves
    y=SA_target
    z=global_tag

    x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
    x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

    if save_data==True:
        np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
        np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
        np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

        np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
        np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
        np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

        np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
        np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
        np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)

    
    print("")
    #del df_num_subset


    del df_num_subset
                                          

In [ ]:
print("saving is done")
now = datetime.now()
print("time at end =", now)